<a href="https://colab.research.google.com/github/yingzi2022/22-Big-Data/blob/main/HW22_s3_rds_us_Kitchen_wo_pwd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.co

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-06 04:18:44--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.12MB/s    in 0.2s    

2022-08-06 04:18:44 (4.12 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in selected tsv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Kitchen_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy/MM/dd")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

## Drop duplicates and incomplete rows

In [ ]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

4880466
4879961
4879961


## Examine the schema

In [ ]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



## Rename columns

In [ ]:
df1 = df.withColumnRenamed("marketplace", "country") \
        .withColumnRenamed("customer_id", "customer_id") \
        .withColumnRenamed("review_id", "review_id") \
        .withColumnRenamed("product_id", "product_id") \
        .withColumnRenamed("product_parent", "product_parent") \
        .withColumnRenamed("product_title", "product_title") \
        .withColumnRenamed("product_category", "product_category") \
        .withColumnRenamed("star_rating", "star_rating") \
        .withColumnRenamed("helpful_votes", "helpful_votes") \
        .withColumnRenamed("total_votes", "total_votes") \
        .withColumnRenamed("vine", "vine") \
        .withColumnRenamed("verified_purchase", "verified_purchase") \
        .withColumnRenamed("review_headline", "review_headline") \
        .withColumnRenamed("review_body", "review_body") \
        .withColumnRenamed("review_date", "review_date")
df1.show(5)

+-------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|country|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|     US|   24197845|R1002B10PD9ZFM|B000LTOCSG|     630072149|Replacement Filte...|         Kitchen|          5|            0|          0|   N|                Y|          Very Nice!|Very inexpensive,...| 2009-07-20|
|     US|   17990743|R1007DQP1BG5WJ|B007M2OHEY|     362955205|Fred & Friends E....|         Kitchen|          5|            0|          

In [ ]:
df1.printSchema()

root
 |-- country: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



## Create a new DataFrame for review info

In [ ]:
# import data time functions
# create table 
from pyspark.sql.functions import *
review_id_table = df1.select(["review_id", "customer_id", "product_id", "product_parent",to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_table.show(5)


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1002B10PD9ZFM|   24197845|B000LTOCSG|     630072149| 2009-07-20|
|R1007DQP1BG5WJ|   17990743|B007M2OHEY|     362955205| 2014-09-12|
|R100AIJ0ZFN8CZ|   33816354|B0001ZYYQU|     331822491| 2007-03-09|
|R100C48B994FWD|   13231777|B00IK59T1S|     744019201| 2014-11-26|
|R100E300CKVFYD|   53006884|B00833DQCU|     298219798| 2013-12-08|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
# Check data column type in the table as needed
# review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
# create table 
products = df1.select(["product_id", "product_title"]).distinct()
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0006H4BF2|WMF Profi Plus Mi...|
|B002PH0LF0|ChefLand Measured...|
|B003QZNABC|Sodastream 1l Car...|
|B00L3NSOJW|Home Value 84 Oun...|
|B000UOARZ4|Fox Run 4091 Upri...|
+----------+--------------------+
only showing top 5 rows



In [ ]:
# Check data column type in the table as needed
# products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [ ]:
#count customer_id
customers = df1.groupBy('customer_id').agg({"customer_id": "count"})
customers = customers.withColumnRenamed("count(customer_id)", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16308995|             2|
|    4709446|             1|
|   31546122|             2|
|   25866981|             7|
|   11537699|             4|
|   50481713|             2|
|   14863908|             2|
|   52964170|             1|
|   51863594|             1|
|   42176175|             5|
|   39258095|             1|
|   52456816|             5|
|   23141101|             3|
|   40302623|             2|
|   18614361|             1|
|   25748567|             2|
|   15185969|             9|
|   30414919|             8|
|   37589671|             1|
|   17018430|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Check data column type in the table as needed
# customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [ ]:
# create table 
vine_table = df1.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1002B10PD9ZFM|          5|            0|          0|   N|
|R1007DQP1BG5WJ|          5|            0|          0|   N|
|R100AIJ0ZFN8CZ|          4|            3|          3|   N|
|R100C48B994FWD|          5|            0|          0|   N|
|R100E300CKVFYD|          1|            3|          3|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [ ]:
# Check data column type in the table as needed
# vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



## Write DataFrame to RDS

In [ ]:
# Configuration for RDS instance
mode="append"
# jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/my_data_class_db"
jdbc_url = "jdbc:postgresql://<end point>:5432/testdb"
config = {"user":"postgres",
          "password": <password>,
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table created above

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to table created above

products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to table created above

customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to table created above

vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)